In [1]:
import pandas
import gzip
from collections import defaultdict
import numpy as np


In [2]:
def readGz(f):
    for l in gzip.open(f):
        yield eval(l)

In [9]:
#baselines.py

training_Ratings = []
training_userRatings = defaultdict(list)


validation_userRatings = defaultdict(list)

num_lines = 0

for l in readGz("train.json.gz"):
    user,business = l['reviewerID'],l['itemID']

    if num_lines < 100000:
        training_Ratings.append(l['rating'])
        training_userRatings[user].append(l['rating'])
        num_lines +=1
    else:
        validation_userRatings[user].append(l['rating'])
        
globalAverage = sum(training_Ratings) / len(training_Ratings)
userAverage = {}
for u in training_userRatings:
    userAverage[u] = sum(training_userRatings[u]) / len(training_userRatings[u])

#find rating mse for training and validation data
training_mse = 0
validation_mse = 0
num_lines2 = 0
for l in readGz("train.json.gz"):
    user,business = l['reviewerID'],l['itemID']
    if user in userAverage:
        prediction = userAverage[user]
    else:
        prediction = globalAverage
    if num_lines2 < 100000:
        training_mse += (l['rating'] - prediction)**2
    if num_lines2 >= 100000:
        validation_mse += (l['rating'] - prediction)**2
    num_lines2 +=1

print("Training Mean Squared Error: " + str(training_mse/100000))
print("Validation Mean Squared Error: " + str(validation_mse/100000))

Training Mean Squared Error: 0.6527804770760693
Validation Mean Squared Error: 1.4976080782955097


In [10]:
#Create training and validation set
training_data = {}
validation_data = {}
 
all_users = list()
all_items= list()
    
split=0
#create training and validation set
for l in readGz("train.json.gz"):
    user,item,rating = l['reviewerID'],l['itemID'],l['rating']
    all_users.append(user)
    all_items.append(item)
    
    if split < 100000:
        training_data[(user,item)] = rating
    else:
        validation_data[(user,item)] = rating
    split += 1    
    

In [ ]:
#baseline gets 1.31393 on test data

In [86]:
#latent factor model
import random
#perform grad descent
n=5
P= np.random.rand(len(all_users),n)
Q= np.random.rand(n, len(all_items))
alpha = globalAverage
beta_u = np.random.rand(len(all_users),1)
index=0
lmbda = .05 #lambda hyperparam
gamma= .005 #gamma hyperparam
iterations = 10000


print("begin gradient descent")
for i in range(iterations):
    if i %1000 ==0:
        print("iteration number " + str(i) )
   # print("iteration number " + str(i))
    grad_q = 0
    grad_p = 0
    grad_a = 0 
    grad_bu = 0
    pair = random.choice(list(training_data.keys()))
    r_ui = training_data[pair]
    (userId,itemId) = pair    
    uIndex = all_users.index(userId)    
    iIndex = all_items.index(itemId)
    summation = 0
    q_val = np.zeros((n, len(all_items)))
    p_val = np.zeros((len(all_users),n))
    for j in range(n):
        summation += np.dot(P[uIndex][j],Q[j][iIndex])
        q_val += Q[j][iIndex]
        p_val += P[uIndex][j]
    grad_a = 2*(r_ui -(alpha + beta_u[uIndex] + summation))*-1
    grad_bu = 2*(r_ui -(alpha + beta_u[uIndex] + summation))*-1
    grad_p = 2*(r_ui -(alpha + beta_u[uIndex] + summation))*-q_val.T
    grad_q = 2*(r_ui -(alpha + beta_u[uIndex] + summation))*-p_val.T

    
    alpha = alpha - gamma*grad_a
    beta_u = beta_u - gamma*grad_bu
    P = P - gamma*grad_p
    Q = Q- gamma*grad_q


begin gradient descent
iteration number 0
iteration number 1000
iteration number 2000
iteration number 3000
iteration number 4000
iteration number 5000
iteration number 6000
iteration number 7000
iteration number 8000
iteration number 9000


In [87]:

training_mse = 0
print("begin calculating training mean squared error")
for pair in training_data.keys():
    real_rating = training_data[pair]
    (userId,itemId) = pair
    userindex = all_users.index(userId)
    itemindex = all_items.index(itemId)
#     print(beta_u.shape)
#     print(P[userindex].shape)
#     print((Q.T)[itemindex].shape)
#     print(np.dot(P[userindex][:],Q[:][itemindex]).shape)
    pred_rating = alpha[0] + beta_u[userindex][0] + np.dot(P[userindex],(Q.T)[itemindex])
    print(pred_rating)
    
    training_mse += (real_rating- pred_rating)**2
print("Training Data mse: "+ str(training_mse/len(training_data.keys())))

print("begin calculating validation mean squared error")

validation_mse = 0
for pair in validation_data.keys():
    real_rating = validation_data[pair]
    (userId,itemId) = pair
    userindex = all_users.index(userId)
    itemindex = all_items.index(itemId)
    pred_rating = alpha[0] + beta_u[userindex][0] + np.dot(P[userindex],(Q.T)[itemindex])
    validation_mse += (real_rating - pred_rating)**2
print("Validation mse: "+ str(validation_mse/len(validation_data.keys())))


begin calculating training mean squared error
4.260380197501361
4.753461233153316
4.193038239969511
4.171362371129759
4.866960110679798
4.08957413523596
4.075506771620124
3.8659855621351165
4.3658161014229835
4.145506780052539
4.35821837600506
3.6836246806558974
4.3731045542466935
4.119611805614173
4.550304316289132
4.170977918604336
4.40100649426054
3.837271385958663
3.6204489666173876
4.670083714763558
4.511298640641882
4.789114727889908
3.8999031059148974
3.6454701456238263
4.314109076660336
4.407396541743981
3.8265280659366216
4.232442892405014
3.839584677676165
4.540886957341696
4.357464355577941
4.115835115666911
4.483783852504355
4.051661914689904
4.723525452721087
4.521227913798701
4.239932111263338
4.6118333328452
4.474626804757417
4.048778295150399
4.1282100347672674
3.880383339624566
4.060925063618813
4.3488050320188885
4.886041402590886
4.574002093481138
3.716632863241058
4.477341402729589
4.682472749629088
3.8441565014186554
4.0142609475380056
4.152651977480645
3.778344451

4.097431536701525
4.297029480107928
4.068731261330381
4.628356981993242
4.23286357066843
4.647193639193194
4.27876136072534
4.484573959848785
3.5591100974756333
4.308883455155982
3.959255483310429
4.305678699067784
3.460041980031469
4.370587729319673
3.8637164546149063
5.127394622560147
4.121969852197313
3.844012070976976
4.592171200623237
3.8663816837101614
4.212675954518146
3.6612585786528875
4.368130111304581
3.75302555183791
3.9065048529034265
3.484115175320229
3.508573042457548
4.198834872595897
4.16671043596836
4.389119845706899
3.9229437021313602
4.6641208104234035
4.0668474738906415
4.136550578665595
3.9891051427894153
4.219587584643938
4.010097097318803
4.252754502794806
3.604432470456468
3.769102787318756
4.578810701369825
4.433722145634185
4.525062604269614
4.136196215881262
4.007690072177074
3.778307371290178
4.714648068139371
4.579703000036716
4.43354405322607
4.5387267720937015
3.5809808422843985
3.8446931794977437
4.27088951582825
4.311283085098322
3.6791771769009425
4.5

5.011075693359718
4.730158231925121
3.8846265140874237
4.549954393155181
4.237271684905531
4.192722933548088
3.7675518280629254
4.2657528432867915
3.9061249422964135
4.592097727623897
4.547537461507064
3.9913502870246123
3.578428723251199
3.4477968713414113
3.877673325445494
3.9025939658920503
3.8414851620479498
4.190781065539307
3.7281930116227224
3.725707027036035
3.649025683714063
4.4879584537496235
4.462648310507547
3.7016496094296034
3.828291030633081
4.112488935894165
4.294304237443998
4.213919964200642
4.097838910280547
4.807210103298101
3.6746031203471463
4.465902136301698
3.5819776055332975
3.9678145115880596
4.633294600784217
4.288285134737881
4.402055640007227
4.327160827839742
4.60919045673854
3.804488809176362
4.26025223974798
3.8696261670611496
3.9990984225335557
3.720840988677104
4.194183893135202
3.7325498466842006
4.533902123391561
3.9463300176547134
4.198963487369932
4.349045225579074
4.181062262377051
4.368488176812862
4.421289250603015
4.603343768185666
4.2343350932

4.079739983513792
3.8969146195114672
4.736458474745336
3.7886506976848606
4.333210408081204
3.9928366242947977
4.6069132509164294
4.190648693457103
3.9826645744575115
3.9322100907592827
3.7127067884741884
3.8712343842348456
4.468696028719271
3.895489078339363
4.559972665043336
4.120091771241104
3.454539227658434
3.583937577412818
4.056894502614889
4.667130043564882
4.209589472696253
3.9429272947056395
3.4764721706960353
4.344200958235522
4.064214347134109
4.116936702821047
4.200144238933578
3.9804320557005712
3.6010120507808363
3.7796886896150084
4.102855576146486
3.8619841220912767
4.489303014295579
4.459965387296694
3.6554210706027743
3.7966829297588265
4.472855324129418
3.6829628359911393
3.6287068383568637
4.258083116510602
4.596664842223701
4.443483163401088
3.8924158889663634
4.2169660018987365
4.076820420799304
3.655023875998225
4.043458692699685
4.381634843975873
4.572941149144568
4.176853461958081
3.629591961754204
3.844125160883601
4.585144043513248
4.055752323895803
4.511587

4.444758388326477
4.089878806654795
4.057810184644188
4.08808185111729
4.076155629750808
4.086633289635799
4.09265405852849
3.5846498533150832
4.391748871357724
4.040656544397982
3.8346082372922874
4.07642896450867
4.13871067599592
4.458929097109307
4.524381106061776
4.225690432417322
3.9455026266840454
3.4066684992203324
4.164677065376353
4.650900928724947
4.411189376928016
3.8228433749286204
4.629241138619657
4.502945117829188
3.9472087583209134
4.652137924010144
4.444367387425537
4.213997790603028
4.268075978243283
4.010361151803196
3.9759366687589326
4.0578163705802055
4.180483770008618
4.262706905911948
3.919571362713723
4.3138503303206255
4.549561130827403
3.705883603704762
4.233148520211426
4.079969310015177
3.6280760117880053
4.664543818868379
3.8294981871609206
3.8027572197980106
4.1806173078095625
4.170738919686522
4.343310759028584
4.182025083256042
3.991184622486654
3.922199332262409
3.8038724457855904
3.6397634904318625
4.256689811671405
3.521845565787753
4.183318089859374

...

4.565708622231353
4.744153317665047
4.485249354488488
4.654755982998045
4.417366259019691
4.146615607540081
4.242603214884345
4.523879993938552
3.93463903976116
4.144973985837905
4.302836900159608
4.42273177185171
4.3563437664948275
4.489469903765663
3.9136782427639467
3.958117729131217
3.969433139801575
3.897931616198914
4.301087768499424
4.2094312767729605
4.251963742645633
4.393327295500857
4.452839493168213
4.03882041635031
4.002915416696095
4.218930309884055
5.071451651730831
4.772965080039169
4.087056314608385
3.8034469836505815
3.699904400842381
3.960082085347246
4.083880572641835
3.747040502415424
4.290492500896346
3.5939413766848562
3.6623293520610725
4.04013697532488
4.512604638445979
4.027536799402676
3.8717990678596506
4.58994780471361
4.338944794815105
4.085252823058
4.6555555925603365
3.584343705497589
3.918466278596022
4.544967256620561
4.273924496000343
4.4537079806441415
4.117485947058813
3.990545385413308
4.7269672511992
3.976277384571065
3.889131230057708
4.714513572

3.8837608368643113
4.664192690335283
3.9178324626977084
4.383358732242683
3.6784915152088047
3.7165252938862285
3.683438195882541
4.361744678140307
4.055847748513159
3.9625282015212293
4.387274039590349
4.126215279894171
3.869244791550209
4.144453849459124
4.729686626672857
3.8766716318276027
4.008907591795907
3.733009278273132
3.9939397200042452
4.599333042788502
3.909238255298826
3.820536779131546
4.106311901068093
4.109822888892312
3.9333389585915586
4.08348730869074
4.446893334692475
4.708421280172003
4.630801704957366
4.61419239637876
3.943870337267428
4.093484057525659
3.596081802182865
3.6407895744279886
3.688517302030113
4.177005620146306
4.382196403656216
4.631415046133896
3.633001210771646
3.550759035834448
3.9353931204628387
4.539046442342913
3.8745856695720584
4.510572124508636
4.123025242968103
3.3973779809222187
4.365725957397193
4.213877982764845
4.013118274543179
3.9567670957390932
3.6582251229233265
4.184635032737378
4.000401949801975
3.72374354196883
3.742816636323600

3.9151982593068557
4.587943805631482
4.207011922771345
4.709337604419426
4.271976327404834
3.508584598924385
4.270090214367754
4.507316033463779
4.489077790054611
4.130192057870188
3.447793818820045
4.337476353705373
3.913166674740558
4.0821081635553025
4.256761004225929
3.875105383403806
3.89553387744023
4.442498094112404
4.3001946913736795
4.405599240599295
4.262696327802279
3.693816968719714
4.745699755843882
3.9456025592520763
4.042728428495153
3.897027921578406
4.113797401651083
4.525058739064628
4.692966106120698
4.205368247892131
4.5756165834725335
4.784012304989793
4.292121773181902
4.09266918556668
4.037173933354232
3.731242048856087
4.020913236701635
3.810315870788214
4.2708038309909515
4.306484271411455
4.88835622463678
4.636979303899798
3.9389117262185254
4.317503914588899
4.441925583664172
4.357094754741233
4.048557825051666
4.631153563198728
3.9301708156136645
3.9032400035241146
3.991613655210113
4.759521777326526
3.9129719175456943
4.720497959980856
3.7118436839859967
4.

3.7199089860727645
3.9361668393273104
4.596263614398285
4.295566192784273
3.9522756041991305
4.0661700548950215
4.326950879017199
3.8520963628535942
4.057332708725013
4.092299827795035
4.315949184263108
3.585794277022334
4.354454894496432
4.391316007479063
4.175454849287123
4.198689543413052
3.8795646525160272
4.358726938467879
3.735831565481758
4.093097163832913
4.25417732170262
4.286021683664382
4.556501086179333
3.8311480556839212
4.342530258507555
4.54614552032339
4.701935171633631
3.475716016122628
3.6056945229760733
3.486511914140365
4.058057963228811
3.754142626879944
4.695440670119818
4.519128258079159
3.328139988495786
4.240493668061168
4.375040509872152
3.803368423674602
4.533366244581319
4.331923306507983
4.312099355262927
4.845686610603332
4.297077352498578
4.123251590381415
4.5286044760263895
4.626478141568676
4.419622976932178
4.229258104028663
3.9906048168674477
4.158111360732444
3.6839082561932437
3.8066789742578977
4.547321160703662
3.7703948652364523
4.218154581735721

Validation mse: 1.3529848505500985


Baseline
Training Mean Squared Error: 0.6527804770760693
Validation Mean Squared Error: 1.4976080782955097

My Results
lambda: .01, gamma .001, iters: 10000, 5 factors
Training Data mse: 1.3465521145092358
begin calculating validation mean squared error
Validation mse: 1.3399293960615437

lambda: .01 gamma .005 iters 10000, 5 factors
Training Data mse: 1.3465207720142207
begin calculating validation mean squared error
Validation mse: 1.3398456298426356


lambda: .01 gamma .01 iters 10000, 5 factors
Training Data mse: 1.386371888971709
begin calculating validation mean squared error
Validation mse: 1.3775317550803412


lambda: .01 gamma .005 iters 10000, 10 factors
Training Data mse: 1.422641247989086
begin calculating validation mean squared error
Validation mse: 1.4126002616984492

lambda: .01 gamma .005 iters 10000, 3 factors
Training Data mse: 1.3488242622374744
begin calculating validation mean squared error
Validation mse: 1.3446991268837358

same as above with 6 latent factors
Training Data mse: 1.3602101491616267
begin calculating validation mean squared error
Validation mse: 1.3520673845715372


lambda: .05 gamma .005 iters 10000, 5 factors
Training Data mse: 1.3626210835902632
begin calculating validation mean squared error
Validation mse: 1.3529848505500985